In [51]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from numpy.linalg import cholesky

## Test 5.1 Normal Simulation PD Input 0 mean - 100,000 simulations, compare input vs output covariance

In [33]:
data_5_1 = pd.read_csv("testfiles/data/test5_1.csv")

def normal_monte_carlo_simulation(mean_vector, covariance_matrix, n_sims, asset_prices:np.array, asset_holdings:np.array, return_type = "arithmetic"):

    portfolio_value = asset_weights.dot(asset_prices)

    #

    np.random.multivariate_normal(mean_vector, covariance_matrix, n_sims)

    
    if return_type == "arithmetic":
        new_prices = asset_prices * (1 + random_returns)
    elif return_type == "geometric":
        portfolio_values = portfolio_value * np.exp(random_returns)
    elif return_type == "brownian":
        portfolio_values = portfolio_value + random_returns



In [37]:
data_5_1

,x1,x2,x3,x4,x5
0,0.084979,0.087586,0.042304,0.008984,0.003876
1,0.087586,0.160485,0.058136,0.012345,0.005326
2,0.042304,0.058136,0.037440,0.005963,0.002573
3,0.008984,0.012345,0.005963,0.001688,0.000546
4,0.003876,0.005326,0.002573,0.000546,0.000314


In [55]:
mean_vector = np.array([0,0,0,0,0])
A = cholesky(data_5_1)
simulated_standard_normal = np.random.multivariate_normal(mean_vector, data_5_1, 2)
transformed_sims = (A.dot(simulated_standard_normal.T) + mean_vector[:, np.newaxis]).T


In [56]:
transformed_sims

array([[ 0.12218575,  0.3835954 ,  0.14835365,  0.02736547,  0.01180554],
       [-0.15599712, -0.33953704, -0.16358798, -0.02841137, -0.01224991]])

In [42]:
norm.rvs(loc=[[0,0,0,0,0]], scale = data_5_1, size=10)

ValueError: size does not match the broadcast shape of the parameters. 10, (1, np.int64(10)), (5, 5)

In [36]:
test_5_1 = pd.read_csv("testfiles/data/testout_5.1.csv")

In [38]:
test_5_1

,x1,x2,x3,x4,x5
0,0.085347,0.087720,0.042334,0.009020,0.003888
1,0.087720,0.160529,0.058067,0.012351,0.005324
2,0.042334,0.058067,0.037468,0.005974,0.002574
3,0.009020,0.012351,0.005974,0.001692,0.000549
4,0.003888,0.005324,0.002574,0.000549,0.000315


## Test 8.1 Normal VaR

In [3]:
data_8_1 = pd.read_csv("testfiles/data/test7_1.csv")

In [4]:
# function from homework 1
def fit_normal_dist_from_data(x: pd.DataFrame):
    mu_vector = x.mean()
    covariance_matrix = x.cov()

    return mu_vector, covariance_matrix

mu_vector, covariance_matrix = fit_normal_dist_from_data(data_8_1)
mean, std = mu_vector.iloc[0], np.sqrt(covariance_matrix.iloc[0,0])


def univariate_normal_VaR(x: pd.DataFrame, alpha = 0.05):
    mu_vector, covariance_matrix = fit_normal_dist_from_data(x)
    mean, std = mu_vector.iloc[0], np.sqrt(covariance_matrix.iloc[0,0])

    abs_VaR = np.abs(norm.ppf(alpha, loc = mean, scale = std))
    rel_VaR = np.abs(norm.ppf(alpha, loc = mean, scale = std) - mean)

    return abs_VaR, rel_VaR

In [8]:
abs_VaR, rel_VaR = univariate_normal_VaR(data_8_1, alpha = 0.05)
test_8_1 = pd.read_csv("testfiles/data/testout8_1.csv")
error_epsilon =1e-12
assert 	abs(abs_VaR - test_8_1.loc[0, "VaR Absolute"]) < error_epsilon, "Absolute value at risk does not match"
assert 	abs(rel_VaR - test_8_1.loc[0, "VaR Diff from Mean"]) < error_epsilon, "Absolute value at risk does not match"

## Test 8.2 VaR T Distribution

In [6]:
data_8_2 = pd.read_csv("testfiles/data/test7_2.csv")

# function from homework 1
from scipy.stats import t
nu, mu, sigma = t.fit(data_8_2)
mu, sigma, nu
alpha = 0.05
# print(t.ppf(alpha, df = nu, loc = mu, scale = sigma))

def univariate_t_VaR(x: pd.DataFrame, alpha = 0.05):
    nu, mu, sigma = t.fit(x)

    abs_VaR = abs(t.ppf(alpha, df = nu, loc = mu, scale = sigma))
    rel_VaR = abs(t.ppf(alpha, df = nu, loc = mu, scale = sigma) - mu)
        
    return abs_VaR, rel_VaR

In [9]:
data_8_2 = pd.read_csv("testfiles/data/test7_2.csv")
abs_VaR, rel_VaR = univariate_t_VaR(data_8_2, alpha = 0.05)

test_8_2 = pd.read_csv("testfiles/data/testout8_2.csv")
error_epsilon = 1e-7
assert 	abs(abs_VaR - test_8_2.loc[0, "VaR Absolute"]) < error_epsilon, "Absolute value at risk does not match"
assert 	abs(rel_VaR - test_8_2.loc[0, "VaR Diff from Mean"]) < error_epsilon, "Absolute value at risk does not match"

## Test 8.3 VaR from Simulation

In [16]:
data_8_3 = pd.read_csv("testfiles/data/test7_2.csv")

,x1
0,0.062695
1,-0.001343
2,0.058816
3,0.074756
4,0.014312
...,...
95,0.083073
96,0.125152
97,0.046132
98,0.036900


In [12]:
np.sort(t.rvs(df=3, loc=0, scale=1, size=10, random_state=0))
# np.floor(3.1)

array([-2.19746122, -1.53369098, -0.92363499, -0.15447375,  0.42911921,
        0.81651664,  1.1574403 ,  1.67969022,  1.85192399,  1.868763  ])

In [86]:
0.99/1.00 - 1

-0.010000000000000009

In [90]:
(-.01)* 1

-0.01

In [32]:
# TODO

def monte_carlo_VaR_sim(x: pd.DataFrame, n_draws, return_type = "arithmetic", alpha = 0.05, seed = 1234):

    if return_type not in ["arithmetic", "geometric", "brownian"]:
        raise ValueError("Returns must be one of arithmetic, geometric, brownian")
    
    # mu, sigma = fit_normal_dist_from_data(x)
    # random_returns = norm.rvs(loc=mu, scale=sigma, size = n_draws, random_state = 0)
    nu, mu, sigma = t.fit(x)
    random_returns = t.rvs(df=nu, loc=mu, scale=sigma, size=n_draws, random_state=seed)

    portfolio_value = 1
    if return_type == "arithmetic":
        portfolio_values = portfolio_value * (1 + random_returns)
    elif return_type == "geometric":
        portfolio_values = portfolio_value * np.exp(random_returns)
    elif return_type == "brownian":
        portfolio_values = portfolio_value + random_returns

    portfolio_returns = portfolio_value - portfolio_values
    sorted_values = np.sort(portfolio_returns)

    alpha_index_low = int(np.floor(alpha * n_draws))
    alpha_index_high = int(np.ceil(alpha * n_draws))

    abs_VaR_low = -sorted_values[alpha_index_low]
    rel_VaR_low = -(sorted_values[alpha_index_low] - np.mean(sorted_values))

    abs_VaR_high = -sorted_values[alpha_index_high]
    rel_VaR_high = -(sorted_values[alpha_index_high] - np.mean(sorted_values))

    abs_VaR = 0.5*(abs_VaR_low + abs_VaR_high)
    rel_VaR = 0.5*(rel_VaR_low + rel_VaR_high)

    return abs_VaR, rel_VaR

    # now/prev - 1 = ret   now - prev = ret * prev -> now = ret * prev + prev

nu, mu, sigma = t.fit(x)
random_returns = t.rvs(df=nu, loc=mu, scale=sigma, size=n_draws, random_state=seed)


monte_carlo_VaR_sim(data_8_3, n_draws=1_000_000, return_type = "arithmetic", alpha=0.05, seed=1234)

(np.float64(0.04156374098539328), np.float64(0.08750725498607156))

In [113]:
test_8_3 = pd.read_csv("testfiles/data/testout8_3.csv")

In [114]:
test_8_3

,VaR Absolute,VaR Diff from Mean
0,0.040212,0.086586
